In [ ]:
import React from 'react';
import { motion } from 'framer-motion';
import { Flame, Beef, Wheat, Droplets } from 'lucide-react';

const CircularProgress = ({ value, max, color, size = 120, strokeWidth = 10 }) => {
  const radius = (size - strokeWidth) / 2;
  const circumference = radius * 2 * Math.PI;
  const progress = Math.min((value / max) * 100, 100);
  const offset = circumference - (progress / 100) * circumference;

  return (
    <div className="relative" style={{ width: size, height: size }}>
      <svg width={size} height={size} className="transform -rotate-90">
        <circle
          cx={size / 2}
          cy={size / 2}
          r={radius}
          fill="none"
          stroke="currentColor"
          strokeWidth={strokeWidth}
          className="text-slate-100"
        />
        <motion.circle
          cx={size / 2}
          cy={size / 2}
          r={radius}
          fill="none"
          stroke={color}
          strokeWidth={strokeWidth}
          strokeLinecap="round"
          strokeDasharray={circumference}
          initial={{ strokeDashoffset: circumference }}
          animate={{ strokeDashoffset: offset }}
          transition={{ duration: 1, ease: 'easeOut' }}
        />
      </svg>
      <div className="absolute inset-0 flex flex-col items-center justify-center">
        <span className="text-2xl font-bold text-slate-800">{value}</span>
        <span className="text-xs text-slate-400">/ {max}</span>
      </div>
    </div>
  );
};

const MacroBar = ({ icon: Icon, label, value, max, color, bgColor }) => {
  const percentage = Math.min((value / max) * 100, 100);
  
  return (
    <div className="flex items-center gap-3">
      <div className={`w-10 h-10 rounded-xl ${bgColor} flex items-center justify-center flex-shrink-0`}>
        <Icon className={`w-5 h-5 ${color}`} />
      </div>
      <div className="flex-1 min-w-0">
        <div className="flex justify-between items-center mb-1">
          <span className="text-sm font-medium text-slate-700">{label}</span>
          <span className="text-sm text-slate-500">{value}g / {max}g</span>
        </div>
        <div className="h-2 bg-slate-100 rounded-full overflow-hidden">
          <motion.div 
            className={`h-full ${color.replace('text-', 'bg-')} rounded-full`}
            initial={{ width: 0 }}
            animate={{ width: `${percentage}%` }}
            transition={{ duration: 0.8, ease: 'easeOut' }}
          />
        </div>
      </div>
    </div>
  );
};

export default function DailyProgress({ todayScans = [], targets }) {
  const totals = (todayScans || []).reduce((acc, scan) => ({
    calories: acc.calories + (scan.calories || 0),
    protein: acc.protein + (scan.protein || 0),
    carbs: acc.carbs + (scan.carbs || 0),
    fats: acc.fats + (scan.fats || 0),
  }), { calories: 0, protein: 0, carbs: 0, fats: 0 });

  const calorieTarget = targets?.daily_calorie_target || 2000;
  const proteinTarget = targets?.daily_protein_target || 150;
  const carbsTarget = targets?.daily_carbs_target || 250;
  const fatsTarget = targets?.daily_fats_target || 65;

  return (
    <div className="bg-white rounded-3xl p-6 shadow-sm border border-slate-100">
      <h3 className="text-lg font-semibold text-slate-800 mb-6">Today's Progress</h3>
      
      <div className="flex flex-col md:flex-row items-center gap-8">
        {/* Calories Circle */}
        <div className="flex flex-col items-center">
          <div className="relative">
            <CircularProgress 
              value={Math.round(totals.calories)} 
              max={calorieTarget} 
              color="#f97316"
              size={140}
              strokeWidth={12}
            />
            <div className="absolute -bottom-1 left-1/2 -translate-x-1/2">
              <div className="bg-gradient-to-r from-orange-400 to-amber-500 text-white text-xs font-medium px-3 py-1 rounded-full flex items-center gap-1">
                <Flame className="w-3 h-3" />
                kcal
              </div>
            </div>
          </div>
          <p className="text-sm text-slate-500 mt-4">
            {calorieTarget - Math.round(totals.calories) > 0 
              ? `${calorieTarget - Math.round(totals.calories)} kcal remaining`
              : 'Goal reached!'
            }
          </p>
        </div>

        {/* Macros */}
        <div className="flex-1 w-full space-y-4">
          <MacroBar 
            icon={Beef} 
            label="Protein" 
            value={Math.round(totals.protein)} 
            max={proteinTarget}
            color="text-rose-500"
            bgColor="bg-rose-50"
          />
          <MacroBar 
            icon={Wheat} 
            label="Carbs" 
            value={Math.round(totals.carbs)} 
            max={carbsTarget}
            color="text-amber-500"
            bgColor="bg-amber-50"
          />
          <MacroBar 
            icon={Droplets} 
            label="Fats" 
            value={Math.round(totals.fats)} 
            max={fatsTarget}
            color="text-blue-500"
            bgColor="bg-blue-50"
          />
        </div>
      </div>
    </div>
  );
}